In [1]:
import pandas as pd
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import wandb
import torch
from sklearn.metrics import accuracy_score, f1_score
import ray
from ray import tune
import os

In [2]:
data = pd.read_csv("./dataset/NewsCategorizer.csv")
train_texts, test_texts, train_labels, test_labels = train_test_split(data['short_description'], data['category'], test_size=0.2, shuffle=True)
train_texts, eval_texts, train_labels, eval_labels = train_test_split(data['short_description'], data['category'], test_size=0.2, shuffle=True)

In [3]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)
eval_labels_encoded = label_encoder.transform(eval_labels)

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', cache_dir='./model')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(data['category'].unique()), cache_dir='./model')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)
eval_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)

In [6]:
train_labels_tensor = torch.tensor(train_labels_encoded)
test_labels_tensor = torch.tensor(test_labels_encoded)
eval_labels_tensor = torch.tensor(eval_labels_encoded)

In [7]:
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = NewsDataset(train_encodings, train_labels_tensor)
test_dataset = NewsDataset(test_encodings, test_labels_tensor)
eval_dataset = NewsDataset(eval_encodings, eval_labels_tensor)

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Get the predicted class by using argmax (for multi-class classification)
    preds = np.argmax(predictions, axis=1)
    
    # Calculate accuracy and F1 score
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")  # Use 'micro', 'macro', or 'weighted' based on the task

    return {
        'accuracy': accuracy,
        'f1': f1,
    }

In [10]:
# def get_training_args(learning_rate, batch_size, warmup_steps):
#     return TrainingArguments(
#         output_dir='./autodl-tmp/results',
#         num_train_epochs=1,
#         per_device_train_batch_size=batch_size,  # Hyperparameter
#         per_device_eval_batch_size=batch_size,   # Hyperparameter
#         warmup_steps=warmup_steps,               # Hyperparameter
#         weight_decay=0.01,
#         logging_dir='./autodl-tmp/logs',
#         logging_steps=10,
#         run_name="cla_epoch_1",
#         save_total_limit=3,
#         eval_strategy="epoch",
#     )

In [11]:
from ray.air import session

In [12]:
def train_fn(config, model, train_dataset, eval_dataset):

    try:
        trial_dir = session.get_trial_dir()  # 例如：~/ray_results/test/trial_xxx/
        output_dir = os.path.join(trial_dir, "results")
    except Exception as e:
        print(f"路径错误: {str(e)}")
        raise
    
    # Update training arguments with the hyperparameters from Ray Tune
    training_args = TrainingArguments(
        run_name = "id_1_epoch_2",
        output_dir=output_dir,
        num_train_epochs=2,  
        
        per_device_train_batch_size=config["batch_size"],  # Hyperparameter from Ray Tune
        per_device_eval_batch_size=config["batch_size"],   # Hyperparameter from Ray Tune
        # warmup_steps=config["warmup_steps"],               # Hyperparameter from Ray Tune
        warmup_steps=500,
        learning_rate=config["learning_rate"],              # Hyperparameter from Ray Tune
        
        weight_decay=0.01,
        logging_dir=os.path.join(trial_dir, "logs"),  
        logging_steps=500,
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        metric_for_best_model="eval_accuracy",
    )

    
    # Initialize the Trainer
    trainer = Trainer(
        model=model, 
        args=training_args, 
        train_dataset=train_dataset, 
        eval_dataset=eval_dataset, 
        compute_metrics=compute_metrics,
    )
    try:
        # Train the model
        trainer.train()
    except Exception as e:
        print(f"训练失败: {str(e)}")
        raise

    try:
    # Evaluate the model
        eval_results = trainer.evaluate()
    except Exception as e:
        print(f"评估失败: {str(e)}")
        raise

    try:
    # Return the evaluation results to Ray Tune
        tune.report(metrics=eval_results)
        trainer.save_model(output_dir)
        tune.report(
            metrics=eval_results,
            checkpoint=tune.Checkpoint.from_directory(output_dir)  # 将模型目录作为检查点
        )
    except Exception as e:
        print(f"报告错误: {str(e)}")
        raise


In [13]:
search_space = {
    "learning_rate": tune.grid_search([1e-5, 2e-5]),
    "batch_size": tune.choice([8, 16]),
    # "warmup_steps": tune.choice([500, 1000, 2000]),
}

In [14]:
wandb.init(project="Mlops-classification", entity="yunchiz-new-york-university")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [15]:
current_dir = os.getcwd()
storage_path = f"file://{current_dir}/ray_results"

train_fn_with_params = tune.with_parameters(train_fn, model=model, train_dataset=train_dataset, eval_dataset=eval_dataset)
ray.init(ignore_reinit_error=True)  # Initialize Ray
analysis = tune.run(
    train_fn_with_params,  # The training function that Ray Tune will use
    config=search_space,  # The search space of hyperparameters
    # resources_per_trial={"cpu": 1, "gpu": 1},
    resources_per_trial={"cpu": 0, "gpu": 1},
    num_samples=2,  # Number of trials (hyperparameter combinations)
    verbose=1,  # Verbosity level of Ray Tune
    storage_path=storage_path,
    name="id_1_epoch_2",
)


(train_fn pid=2241) wandb: Currently logged in as: yunchiz (yunchiz-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
(train_fn pid=2241) wandb: Tracking run with wandb version 0.19.9
(train_fn pid=2241) wandb: Run data is saved locally in /tmp/ray/session_2025-04-15_02-17-47_690880_1125/artifacts/2025-04-15_02-17-55/id_1_epoch_2/working_dirs/train_fn_c91b5_00000_0_batch_size=16,learning_rate=0.0000_2025-04-15_02-17-55/wandb/run-20250415_021809-h3bq80dx
(train_fn pid=2241) wandb: Run `wandb offline` to turn off syncing.
(train_fn pid=2241) wandb: Syncing run id_1_epoch_2
(train_fn pid=2241) wandb: ⭐️ View project at https://wandb.ai/yunchiz-new-york-university/huggingface
(train_fn pid=2241) wandb: 🚀 View run at https://wandb.ai/yunchiz-new-york-university/huggingface/runs/h3bq80dx
 10%|▉         | 498/5000 [00:49<07:22, 10.16it/s]


(train_fn pid=2241) {'loss': 1.9581, 'grad_norm': 8.756377220153809, 'learning_rate': 9.980000000000001e-06, 'epoch': 0.2}


 20%|█▉        | 998/5000 [01:38<06:32, 10.19it/s]


(train_fn pid=2241) {'loss': 1.0736, 'grad_norm': 6.7667694091796875, 'learning_rate': 8.891111111111111e-06, 'epoch': 0.4}


 30%|██▉       | 1498/5000 [02:27<05:44, 10.15it/s]


...

  0%|          | 0/1250 [00:00<?, ?it/s]
(train_fn pid=3366) 
  1%|          | 8/1250 [00:00<00:16, 74.52it/s]
(train_fn pid=3366) 
  1%|▏         | 16/1250 [00:00<00:18, 68.17it/s]
(train_fn pid=3366) 
  2%|▏         | 23/1250 [00:00<00:18, 66.41it/s]
(train_fn pid=3366) 
  2%|▏         | 30/1250 [00:00<00:18, 65.61it/s]
(train_fn pid=3366) 
  3%|▎         | 37/1250 [00:00<00:18, 65.09it/s]
(train_fn pid=3366) 
  4%|▎         | 44/1250 [00:00<00:18, 64.81it/s]
(train_fn pid=3366) 
  4%|▍         | 51/1250 [00:00<00:18, 64.77it/s]
(train_fn pid=3366) 
  5%|▍         | 58/1250 [00:00<00:18, 64.10it/s]
(train_fn pid=3366) 
  5%|▌         | 65/1250 [00:00<00:18, 64.13it/s]
(train_fn pid=3366) 
  6%|▌         | 72/1250 [00:01<00:18, 64.15it/s]
(train_fn pid=3366) 
  6%|▋         | 79/1250 [00:01<00:18, 64.38it/s]
(train_fn pid=3366) 
  7%|▋         | 86/1250 [00:01<00:18, 64.50it/s]
(train_fn pid=3366) 
  7%|▋         | 93/1250 [00:01<00:17, 64.47it/s]
(train_fn pid=3366) 
  8%|▊         |

(train_fn pid=3366) {'eval_loss': 5.5601372718811035, 'eval_accuracy': 0.0988, 'eval_f1': 0.09878797028181915, 'eval_runtime': 19.7828, 'eval_samples_per_second': 505.488, 'eval_steps_per_second': 63.186, 'epoch': 2.0}


                                                     
100%|██████████| 1250/1250 [00:19<00:00, 64.46it/s]
                                                   


(train_fn pid=3366) {'train_runtime': 600.9953, 'train_samples_per_second': 133.113, 'train_steps_per_second': 16.639, 'train_loss': 0.7672928863525391, 'epoch': 2.0}


100%|██████████| 1250/1250 [00:19<00:00, 63.94it/s]
(train_fn pid=3366) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/autodl-tmp/ray_results/id_1_epoch_2/train_fn_c91b5_00003_3_batch_size=8,learning_rate=0.0000_2025-04-15_02-17-55/checkpoint_000000)
2025-04-15 02:59:31,061	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/autodl-tmp/ray_results/id_1_epoch_2' in 24.7607s.
2025-04-15 02:59:31,071	INFO tune.py:1041 -- Total run time: 2496.07 seconds (2471.26 seconds for the tuning loop).


In [16]:
best_trial = analysis.get_best_trial(metric="eval_accuracy", mode="max")

# 获取检查点路径（通过 checkpoint 属性）
best_checkpoint = best_trial.checkpoint
best_checkpoint_dir = best_checkpoint.to_directory()  # 提取检查点目录
print(f"最佳模型路径：{best_checkpoint_dir}")

# 加载模型
from transformers import AutoModel
best_model = DistilBertForSequenceClassification.from_pretrained(best_checkpoint_dir)

最佳模型路径：/tmp/checkpoint_tmp_32a01f50c7a4421fb8fceebea21d4f87


In [17]:
trainer = Trainer(
    model=best_model,
    args=TrainingArguments(output_dir="./tmp"),  # 临时目录，仅用于预测
)

predictions = trainer.predict(test_dataset)
predictions_logits = predictions.predictions
predicted_labels = np.argmax(predictions_logits, axis=1)

accuracy = accuracy_score(test_labels_encoded, predicted_labels)
f1 = f1_score(test_labels_encoded, predicted_labels, average="macro")

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score (Macro Average): {f1:.4f}")
wandb.finish()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Accuracy: 0.8056
F1 Score (Macro Average): 0.8055


test/loss,▁
test/model_preparation_time,▁
test/runtime,▁
test/samples_per_second,▁
test/steps_per_second,▁
train/global_step,▁
test/loss,0.64161
test/model_preparation_time,0.0014
test/runtime,19.6133
test/samples_per_second,509.858
test/steps_per_second,63.732
